In [ ]:
import plotly.express as px
import numpy as np
from plotly.subplots import make_subplots
from scipy.ndimage import uniform_filter1d
import plotly.graph_objects as go
from toolkit.common.constants import *
from toolkit.battery_model.battery_model import IRBatteryModel, ParallelGroup, BatteryPack
from toolkit.motor.motor import Motor

In [ ]:
# Create cells for the pack
cell = IRBatteryModel(3.9, 2000, 0.027)  # Example cell parameters
parallel_cells = 7  # 7p configuration
group = ParallelGroup(cell, parallel_cells)
groups_in_series = [group for _ in range(96)]  # 96s configuration

pack = BatteryPack(groups_in_series)

emrax_228_mv = Motor('EMRAX 228 MV')
emrax_228_mv.Ld = 0.000076
emrax_228_mv.Lq = 0.000079
emrax_228_mv.Rs = 0.0071
emrax_228_mv.max_phase_current = 480.0
emrax_228_mv.max_rate = 6500.0
emrax_228_mv.Fl = 0.0355
emrax_228_mv.poles = 10.0
cur_motor = emrax_228_mv

In [ ]:
cell_voltages = np.linspace(2.95, 4.2, 100)
pack_voltage_no_load = np.zeros(len(cell_voltages))
pack_voltages = np.zeros(len(cell_voltages))
pack_powers = np.zeros(len(cell_voltages))
pack_currents = np.zeros(len(cell_voltages))
power_lim = 55000 # 55kW

for i, voltage in enumerate(cell_voltages):
    for group in pack.groups:
        group.cell.voltage = voltage
    pack_voltage_no_load[i] = pack.total_voltage()
    pack_voltages[i], pack_currents[i], pack_powers[i] = pack.get_pack_at_power(power_lim)

fig = go.Figure()
fig.add_trace(go.Scatter(x=cell_voltages, y=pack_voltages))
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=cell_voltages, y=pack_voltage_no_load))
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=cell_voltages, y=pack_powers))
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=pack_voltage_no_load, y=pack_voltages))
fig.show()

In [ ]:
speed_range = np.linspace(100, 6000, 100)

# determine the max torque at each speed while respecting the power limit
# we do this by converging on the torque that equals the power we want
# this is done by using the bisection method
torques = np.zeros((len(speed_range), len(cell_voltages)))
wbases = np.zeros(len(cell_voltages))
pows = np.zeros((len(speed_range), len(cell_voltages)))
powers = np.zeros((len(speed_range), len(cell_voltages)))
ids = np.zeros((len(speed_range), len(cell_voltages)))
iqs = np.zeros((len(speed_range), len(cell_voltages)))
vds = np.zeros((len(speed_range), len(cell_voltages)))
vqs = np.zeros((len(speed_range), len(cell_voltages)))
iters = np.zeros((len(speed_range), len(cell_voltages)))
motor_states = np.zeros((len(speed_range), len(cell_voltages)))
print(pack_voltages)
for i, speed in enumerate(speed_range):
    for j, voltage in enumerate(pack_voltages):
        power = 0
        low = 0
        high = cur_motor.max_torque
        t = 25
        t_prev = 0
        k = 0
        while abs(power - power_lim) > 1 and abs(t - cur_motor.max_torque) > 0.1 and abs(t - t_prev) > 0.05 and k < 100:
            w = (speed * 2 * np.pi) / 60
            Id, Iq, t_out, wbase, v_max, o = cur_motor.get_qd_currents(w, t, voltage, use_mtpa=False)
            v_d, v_q = cur_motor.get_voltages(w, Id, Iq)
            power = cur_motor.calculate_power(w, Id, Iq)
            # print(f"{power:.3f} {power_lim:.2f} t:{t:.3f} t_out:{t_out:.3f} l:{low:.2f} h:{high:.2f} v:{voltage:.2f} s:{speed:.2f} Id:{Id:.2f} Iq:{Iq:.2f} v_d:{v_d:.2f} v_q:{v_q:.2f} wbase:{wbase:.2f}")
            nt = min(t, t_out)
            if power > power_lim or t-t_out > (high-low)/20:
                high = (t + high) / 2
                # print("up")
            else:
                low = (nt + low) / 2
                # print("down")
            t_prev = t
            t = (low + high) / 2
            k += 1
        torques[i, j] = t
        pows[i, j] = t * speed * 2 * np.pi / 60
        powers[i, j] = power
        ids[i, j] = Id
        iqs[i, j] = Iq
        vds[i, j] = v_d
        vqs[i, j] = v_q
        iters[i, j] = k
        motor_states[i, j] = o
        if i == 0:
            wbases[j] = (wbase * 60 / (2 * np.pi))
            
torques[np.isnan(torques)] = 0


# find transition lines
diffs = np.diff(motor_states, axis=0)
# there are 3 transition states, 0->1 entering field weakening, 1->2 Id limit, 2->3 Torque null
def find_transitions(state: int):
    inds = np.argwhere((diffs==1) & (motor_states[:-1,:]==state))
    inds = inds[np.argsort(inds[:,1])]
    return pack_voltage_no_load[inds[:,1]], speed_range[inds[:,0]]

efw_v, efw_w = find_transitions(0)
il_v, il_w = find_transitions(1)
tn_v, tn_w = find_transitions(2)

def plot_wbase(fig: go.Figure):
    fig.add_trace(go.Scatter(x=efw_v, y=efw_w, mode='lines', name="Field Weakening"))
    fig.add_trace(go.Scatter(x=il_v, y=il_w, mode='lines', name="Id Limit"))
    fig.add_trace(go.Scatter(x=tn_v, y=tn_w, mode='lines', name="Bus Voltage Limit"))

def plot_param(heat, name: str):
    fig = go.Figure()
    fig.add_trace(go.Heatmap(x=pack_voltage_no_load, y=speed_range, z=heat))
    plot_wbase(fig)
    fig.update_layout(template="plotly_dark", title_text=name, xaxis_title="Voltage (V)", yaxis_title="Speed (RPM)", legend=dict(orientation="h", yanchor="bottom", y=0.02, xanchor="right", x=1), height=1000, width=1000)
    fig.show()

# make a heatmap of the torque vs speed
plot_param(torques, "Torque (Nm) respecting Power Limit")

# make a heatmap of the mechanical power vs speed
plot_param(pows, "Mechanical Power (kW)")

# make a heatmap of the power vs speed
plot_param(powers, "Electrical Power (kW)")

# make a heatmap of the Id vs speed
plot_param(ids, "Id Current (A)")

# make a heatmap of the Iq vs speed
plot_param(iqs, "Iq Current (A)")

# make a heatmap of the v_d vs speed
plot_param(vds, "Vd Voltage (V)")

# make a heatmap of the v_q vs speed
plot_param(vqs, "Vq Voltage (V)")

# make a heatmap of the power vs speed
plot_param(100*(powers-pows)/powers, "Percent Electrical Inefficiency")

# make a heatmap of the iters vs speed
plot_param(iters, "Convergence Iterations")

# make a heatmap of the motor states vs speed
plot_param(motor_states, "Motor States 0=MTPA, 1=Field Weakening, 2=Id Limited, 3=Null Torque Regime")

In [ ]:
# make the c description of the LUT
output_str = f"const float4 POWER_LIM_LUT[{len(speed_range)}][{len(cell_voltages)}] = {{\n"
for i, speed in enumerate(speed_range):
    output_str += f"\t{{"
    for j, voltage in enumerate(pack_voltages):
        output_str += f"{torques[i, j]:.2f}, "
    output_str = output_str[:-2]
    output_str += f"}},\n"
output_str = output_str[:-2]
output_str += f"}};\n"
output_str += f"const float4 V_MIN = {pack_voltage_no_load[0]:.3f};\n"
output_str += f"const float4 V_MAX = {pack_voltage_no_load[-1]:.3f};\n"
output_str += f"const sbyte4 S_MIN = {int(speed_range[0])};\n"
output_str += f"const sbyte4 S_MAX = {int(speed_range[-1])};\n"
output_str += f"const ubyte1 NUM_V = {len(cell_voltages)};\n"
output_str += f"const ubyte1 NUM_S = {len(speed_range)};\n"

print(output_str)

In [ ]:
emrax_228_mv = Motor('EMRAX 228 MV')
emrax_228_mv.Ld = 0.000076
emrax_228_mv.Lq = 0.000079
emrax_228_mv.Rs = 0.0071
emrax_228_mv.max_phase_current = 450.0
emrax_228_mv.max_rate = 6500.0
emrax_228_mv.Fl = 0.0355
emrax_228_mv.poles = 10.0
# cur_motor = emrax_228_mv

In [ ]:

# import the csv of data 
import pandas as pd
import numpy as np
from scipy.signal import filtfilt, butter, lfilter
from math import ceil
df = pd.read_csv('../Data/logs/20231022-0910604.csv')
# throw away the first row because it is the header
df = df.iloc[1:]
# print(df)

def interpolate_lut(voltage, speed):
    # do a bicubic interpolation to get the torque at the given voltage and speed
    v_ind = ((voltage - pack_voltage_no_load[0]) / (pack_voltage_no_load[-1] - pack_voltage_no_load[0])) * (len(pack_voltage_no_load) - 1)
    vu_ind = ceil(v_ind)
    vd_ind = max(vu_ind - 1, 0)
    v_shift = v_ind - vu_ind + 1
    s_ind = ((speed - speed_range[0]) / (speed_range[-1] - speed_range[0])) * (len(speed_range) - 1)
    su_ind = ceil(s_ind)
    sd_ind = max(su_ind - 1, 0)
    s_shift = s_ind - su_ind + 1
    a, b, c, d = v_shift * s_shift, s_shift * (1 - v_shift), v_shift * (1 - s_shift), (1 - s_shift) * (1 - v_shift)
    return (torques[su_ind, vu_ind] * a) + (torques[su_ind, vd_ind] * b) + (torques[sd_ind, vu_ind] * c) + (torques[sd_ind, vd_ind] * d)


# for each row, calculate the power
powers = []
powers_u = []
i_d, i_q = [], []
tor = []
nlv = []
os = []
lim_tq = []
ir = pack.total_internal_resistance()
print(ir)
b, a = butter(8, 0.0625)
df['Filtered Voltage'] = filtfilt(b, a, df['MCM DC Bus Voltage'].copy())
df['Filtered Speed'] = filtfilt(b, a, df['MCM Motor Speed'].copy())

for i, row in df.iterrows():
    voltage = df['Filtered Voltage'][i]
    nv_voltage = df['MCM DC Bus Voltage'][i] + ir * df['MCM DC Bus Current'][i]
    speed = df['Filtered Speed'][i]
    w = (speed * 2 * np.pi/60)
    t = df['MCM Torque Command'][i]
    limit_torque = interpolate_lut(nv_voltage, speed)
    t = min(t, limit_torque)
    power = cur_motor.calculate_power(w, df['MCM Id'][i], df['MCM Iq'][i])
    Id, Iq, t_out, wbase, v_max, o = cur_motor.get_qd_currents(w, t, voltage, use_mtpa=True)
    v_d, v_q = cur_motor.get_voltages(w, Id, Iq)
    power_u = cur_motor.calculate_power(w, Id, Iq)
    # print(f"{power:.3f} {power_u:.3f} {t_out:.3f} {Id:.2f} {Iq:.2f} {v_d:.2f} {v_q:.2f} {(wbase * 60 / (2 * np.pi)):.2f} {v_max:.2f}")
    powers.append(power)
    i_d.append(Id)
    i_q.append(Iq)
    tor.append(t_out)
    nlv.append(nv_voltage)
    powers_u.append(power_u)
    os.append(o)
    lim_tq.append(limit_torque)
df['power'] = powers
df['power_u'] = powers_u
df['Id'] = i_d
df['Iq'] = i_q
df['Torque'] = tor
df['NV Voltage'] = nlv
df['Operating Mode'] = os
df['Limit Torque'] = lim_tq
df['Filtered NV Voltage'] = filtfilt(b, a, df['NV Voltage'].copy())


fig = make_subplots(rows=7, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(y=df['MCM DC Bus Voltage'] * df['MCM DC Bus Current'], x=df['Time'], name='True'), row=1, col=1)
fig.add_trace(go.Scatter(y=df['power'], x=df['Time'], name='Real Power From Currents'), row=1, col=1)
fig.add_trace(go.Scatter(y=df['power_u'], x=df['Time'], name='Sim'), row=1, col=1)
fig.add_trace(go.Scatter(y=df['MCM Torque Feedback'] * df['MCM Motor Speed'] * 2 * np.pi/60, x=df['Time'], name='Mechanical Torque'), row=1, col=1)
fig.add_trace(go.Scatter(y=df['Torque'] * df['MCM Motor Speed'] * 2 * np.pi/60, x=df['Time'], name='Mechanical Torque sim'), row=1, col=1)

fig.add_trace(go.Scatter(y=df['MCM Id'], x=df['Time'], name='Id'), row=2, col=1)
fig.add_trace(go.Scatter(y=df['MCM Iq'], x=df['Time'], name='Iq'), row=2, col=1)
fig.add_trace(go.Scatter(y=df['Id'], x=df['Time'], name='Id sim'), row=2, col=1)
fig.add_trace(go.Scatter(y=df['Iq'], x=df['Time'], name='Iq sim'), row=2, col=1)


fig.add_trace(go.Scatter(y=df['MCM Torque Command'], x=df['Time'], name='Torque'), row=3, col=1)
fig.add_trace(go.Scatter(y=df['MCM Torque Feedback'], x=df['Time'], name='Torque Feedback'), row=3, col=1)
fig.add_trace(go.Scatter(y=df['Torque'], x=df['Time'], name='Torque sim'), row=3, col=1)
fig.add_trace(go.Scatter(y=df['Limit Torque'], x=df['Time'], name='Limit Torque'), row=3, col=1)


fig.add_trace(go.Scatter(y=df['NV Voltage'], x=df['Time'], name='NV Voltage'), row=4, col=1)
fig.add_trace(go.Scatter(y=df['Filtered NV Voltage'], x=df['Time'], name='Filtered NV Voltage'), row=4, col=1)
fig.add_trace(go.Scatter(y=df['MCM DC Bus Voltage'], x=df['Time'], name='Voltage'), row=4, col=1)
fig.add_trace(go.Scatter(y=df['Filtered Voltage'], x=df['Time'], name='Filtered Voltage'), row=4, col=1)


fig.add_trace(go.Scatter(y=df['MCM DC Bus Current'], x=df['Time'], name='Current'), row=5, col=1)
fig.add_trace(go.Scatter(y=df['power_u']/df['Filtered Voltage'], x=df['Time'], name='Sim Current'), row=5, col=1)


fig.add_trace(go.Scatter(y=df['Filtered Speed'], x=df['Time'], name='Filtered Speed'), row=6, col=1)
fig.add_trace(go.Scatter(y=df['MCM Motor Speed'], x=df['Time'], name='Speed'), row=6, col=1)

fig.add_trace(go.Scatter(y=df['Operating Mode'], x=df['Time'], name='Operating Mode'), row=7, col=1)

fig.update_layout(template="plotly_dark", title_text="Power (W)", height=1200, width=1000)


fig.show()
